# Introduction
Music is an important artifact of culture and society. Thus, analyzing song data can reveal a lot about the societal trends. Music consumption, specifically, tells us how society.

# The Dataset
To aid in our investigation, we use [Dhruvil Dave's dataset](https://www.kaggle.com/datasets/dhruvildave/billboard-the-hot-100-songs/) on songs from the Billboard Hot 100, ranging from the year 1958 to 2021.

All dataset and other intermediary data are stored in the `/data` folder in the root directory of this project. Let's move up a directory so that we have access to the data we're using.

In [1]:
%cd ..

d:\Y3T1\CSMODEL\notebooks\exploring-billboard-data


Now, we can load our dataset.

In [2]:
import pandas as pd
charts_df = pd.read_csv('data/charts.csv')

## Exploring our variables

Now, we have a DataFrame object we can work with! Let us explore the observations and variables in our dataset.

In [3]:
charts_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330087 entries, 0 to 330086
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   date            330087 non-null  object 
 1   rank            330087 non-null  int64  
 2   song            330087 non-null  object 
 3   artist          330087 non-null  object 
 4   last-week       297775 non-null  float64
 5   peak-rank       330087 non-null  int64  
 6   weeks-on-board  330087 non-null  int64  
dtypes: float64(1), int64(3), object(3)
memory usage: 17.6+ MB


We have 330 087 observations and six variables in our dataset. The variables and their description are as follows:
- **date**: date of the chart
- **rank**: rank of the song
- **song**: song title
- **artist**: artist name
- **last-week**: rank of the song in the preceding week
- **peak-rank**: the highest rank that the song historically charted
- **weeks-on-board**: how many weeks the song is charting up to the point of the record (does not have to be consecutive)

The `date`, `song` and `artist` variables are object types (or string types) while the rest are numeric types.

In [4]:
charts_df['date'] = pd.to_datetime(charts_df['date'])
charts_df

,date,rank,song,artist,last-week,peak-rank,weeks-on-board
0,2021-11-06,1,Easy On Me,Adele,1.0,1,3
1,2021-11-06,2,Stay,The Kid LAROI & Justin Bieber,2.0,1,16
2,2021-11-06,3,Industry Baby,Lil Nas X & Jack Harlow,3.0,1,14
3,2021-11-06,4,Fancy Like,Walker Hayes,4.0,3,19
4,2021-11-06,5,Bad Habits,Ed Sheeran,5.0,2,18
...,...,...,...,...,...,...,...
330082,1958-08-04,96,Over And Over,Thurston Harris,NaN,96,1
330083,1958-08-04,97,I Believe In You,Robert & Johnny,NaN,97,1
330084,1958-08-04,98,Little Serenade,The Ames Brothers,NaN,98,1
330085,1958-08-04,99,I'll Get By (As Long As I Have You),Billy Williams,NaN,99,1


### Handling null and duplicate variables
Out of all the variables, the `last-week` variable contains some non-null values as seen in the output for `charts_df.info()`. We can see below how many actual null values we have (32 312 rows).

In [5]:
charts_df['last-week'][charts_df['last-week'].isnull()]

26       NaN
27       NaN
60       NaN
68       NaN
78       NaN
          ..
330082   NaN
330083   NaN
330084   NaN
330085   NaN
330086   NaN
Name: last-week, Length: 32312, dtype: float64

This is expected, however. These null values simply mean that the song entry is new to the charts which corresponds to no record for last week.

For duplicates, the variables `date`, `rank` and other similar chart variables are expected to duplicate. Artists may also have multiple charting songs throughout their career. Songs may chart multiple weeks as well.

However, for the sake of analysis, we might need to create a single entry for each song instead and extract the most relevant features: the `peak-rank` and the maximum value for `weeks-on-board` of that song. 

In this way, we trim our dataset and remove extraneous fluff that may hinder our analysis.

But before we remove the duplicates, we must consider engineering more features. I will explain in a while why we must do feature engineering before handling duplicates.

### Feature engineering
Since we're working with dataset that pertains to cultural trends, one interesting feature we might explore is the decade. We can bin these dates to certain years and decades of the charting.

Here we extract the year of the charting period of a given entry.

In [6]:
year_series = charts_df['date'].dt.year.apply(lambda x: int(x))
year_series.name = 'year'
year_series

0         2021
1         2021
2         2021
3         2021
4         2021
          ... 
330082    1958
330083    1958
330084    1958
330085    1958
330086    1958
Name: year, Length: 330087, dtype: int64

For the analysis to be more helpful, it could benefit from putting the years to specific bins of decades as well.

In [7]:
decade_series = (year_series // 10 * 10).apply(lambda x: int(x))
decade_series.name = 'decade'

We then concatenate the series of new features to our cleaned `DataFrame`.

In [8]:
cleaned_charts_df = pd.concat([charts_df, year_series, decade_series], axis='columns')
cleaned_charts_df

,date,rank,song,artist,last-week,peak-rank,weeks-on-board,year,decade
0,2021-11-06,1,Easy On Me,Adele,1.0,1,3,2021,2020
1,2021-11-06,2,Stay,The Kid LAROI & Justin Bieber,2.0,1,16,2021,2020
2,2021-11-06,3,Industry Baby,Lil Nas X & Jack Harlow,3.0,1,14,2021,2020
3,2021-11-06,4,Fancy Like,Walker Hayes,4.0,3,19,2021,2020
4,2021-11-06,5,Bad Habits,Ed Sheeran,5.0,2,18,2021,2020
...,...,...,...,...,...,...,...,...,...
330082,1958-08-04,96,Over And Over,Thurston Harris,NaN,96,1,1958,1950
330083,1958-08-04,97,I Believe In You,Robert & Johnny,NaN,97,1,1958,1950
330084,1958-08-04,98,Little Serenade,The Ames Brothers,NaN,98,1,1958,1950
330085,1958-08-04,99,I'll Get By (As Long As I Have You),Billy Williams,NaN,99,1,1958,1950


The years in the decades 2020s and 1950s are not of standard length since the dataset only involves the one or two years in that decades. To extract useful comparison between each decade, we can limit our range.

In [9]:
cleaned_charts_df = cleaned_charts_df.query('decade < 2020 & decade > 1950')
cleaned_charts_df

,date,rank,song,artist,last-week,peak-rank,weeks-on-board,year,decade
9700,2019-12-28,1,All I Want For Christmas Is You,Mariah Carey,1.0,1,36,2019,2010
9701,2019-12-28,2,Rockin' Around The Christmas Tree,Brenda Lee,3.0,2,31,2019,2010
9702,2019-12-28,3,Circles,Post Malone,2.0,1,16,2019,2010
9703,2019-12-28,4,Roxanne,Arizona Zervas,6.0,4,7,2019,2010
9704,2019-12-28,5,Futsal Shuffle 2020,Lil Uzi Vert,NaN,5,1,2019,2010
...,...,...,...,...,...,...,...,...,...
322682,1960-01-04,96,Deck Of Cards,Wink Martindale,68.0,7,17,1960,1960
322683,1960-01-04,97,One Mint Julep,Chet Atkins,NaN,97,1,1960,1960
322684,1960-01-04,98,Happy Anniversary,Jane Morgan,57.0,57,9,1960,1960
322685,1960-01-04,99,Smokie-Part 2,Bill Doggett,NaN,99,1,1960,1960


Remember we have duplicate entries for each time a particular song enters the charts. However, it might be useful to have the most important "summary" of the song. This would be the max peak (highest peak throughout the song's lifetime in the charts) and the total weeks on board.

In [10]:
longest_running_charting = cleaned_charts_df.groupby(['song', 'artist', 'decade'], as_index=False).aggregate('max')[['song', 'artist', 'decade', 'weeks-on-board']]
top_rank = cleaned_charts_df.groupby(['song', 'artist', 'decade'], as_index=False).aggregate('min')['rank']
cleaned_charts_df = pd.concat([longest_running_charting, top_rank], axis='columns')
cleaned_charts_df.sort_values(axis='rows', by='decade')

,song,artist,decade,weeks-on-board,rank
6726,Fancy,Bobbie Gentry,1960,6,53
15301,Making Every Minute Count,Spanky And Our Gang,1960,7,31
23663,The One On The Right Is On The Left,Johnny Cash,1960,6,46
15305,Making Memories,Frankie Laine,1960,8,35
15308,Malagueña,Connie Francis,1960,9,42
...,...,...,...,...,...
22637,Tap,NAV Featuring Meek Mill,2010,10,87
22632,Talladega,Eric Church,2010,20,43
9469,"Hold On, We're Going Home",Christina Grimmie,2010,1,74
9473,Hold Tight,Justin Bieber,2010,1,29


However, the chart data alone might not have the most groundbreaking insights. We can augment this primary dataset with another to broaden our investigation.

In [11]:
tracks_df = pd.read_csv('data/tracks.csv')
tracks_df

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,35iwgR4jXetI318WEWsa1Q,Carve,6,126903,0,['Uli'],['45tIt06XoI0Iio4LBEVpls'],1922-02-22,0.645,0.4450,0,-13.338,1,0.4510,0.674,0.744000,0.1510,0.1270,104.851,3
1,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,0,98200,0,['Fernando Pessoa'],['14jtPCOoNZwquk5wd9DxrY'],1922-06-01,0.695,0.2630,0,-22.136,1,0.9570,0.797,0.000000,0.1480,0.6550,102.009,1
2,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,0,181640,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.434,0.1770,1,-21.180,1,0.0512,0.994,0.021800,0.2120,0.4570,130.418,5
3,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,0,176907,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.321,0.0946,7,-27.961,1,0.0504,0.995,0.918000,0.1040,0.3970,169.980,3
4,08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,0,163080,0,['Dick Haymes'],['3BiJGZsyX9sJchTqcSA7Su'],1922,0.402,0.1580,3,-16.900,0,0.0390,0.989,0.130000,0.3110,0.1960,103.220,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
586667,5rgu12WBIHQtvej2MdHSH0,云与海,50,258267,0,['阿YueYue'],['1QLBXKM5GCpyQQSVMNZqrZ'],2020-09-26,0.560,0.5180,0,-7.471,0,0.0292,0.785,0.000000,0.0648,0.2110,131.896,4
586668,0NuWgxEp51CutD2pJoF4OM,blind,72,153293,0,['ROLE MODEL'],['1dy5WNgIKQU6ezkpZs4y8z'],2020-10-21,0.765,0.6630,0,-5.223,1,0.0652,0.141,0.000297,0.0924,0.6860,150.091,4
586669,27Y1N4Q4U3EfDU5Ubw8ws2,What They'll Say About Us,70,187601,0,['FINNEAS'],['37M5pPGs6V1fchFJSgCguX'],2020-09-02,0.535,0.3140,7,-12.823,0,0.0408,0.895,0.000150,0.0874,0.0663,145.095,4
586670,45XJsGpFTyzbzeWK8VzR8S,A Day At A Time,58,142003,0,"['Gentle Bones', 'Clara Benin']","['4jGPdu95icCKVF31CcFKbS', '5ebPSE9YI5aLeZ1Z2g...",2021-03-05,0.696,0.6150,10,-6.212,1,0.0345,0.206,0.000003,0.3050,0.4380,90.029,4


[Yamac Eren's dataset](https://www.kaggle.com/datasets/yamaerenay/spotify-dataset-19212020-600k-tracks) consists of Spotify data of around 600 000 tracks. Like the Billboard Hot 100, this dataset can be generated from public data given by Spotify. Specifically, it can be gathered through the Spotify API.

It consists of the following important features: 
- **name**: the title of the song
- **duration_ms**: the length of the song in millisecond
- **artists**: an array of artists featured in the song
- **danceability**: "how suitable a track is for dancing based on a combination of musical elements"
- **energy**: "represents a perceptual measure of intensity and activity"
- **loudness**: "overall loudness of a track in decibels"
- **acousticness**: "confidence measure from 0.0 to 1.0 of whether the track is acoustic"
- **valence**: "describing the musical positiveness conveyed by a track"

In [12]:
tracks_df.head()

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,35iwgR4jXetI318WEWsa1Q,Carve,6,126903,0,['Uli'],['45tIt06XoI0Iio4LBEVpls'],1922-02-22,0.645,0.4450,0,-13.338,1,0.4510,0.674,0.7440,0.151,0.127,104.851,3
1,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,0,98200,0,['Fernando Pessoa'],['14jtPCOoNZwquk5wd9DxrY'],1922-06-01,0.695,0.2630,0,-22.136,1,0.9570,0.797,0.0000,0.148,0.655,102.009,1
2,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,0,181640,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.434,0.1770,1,-21.180,1,0.0512,0.994,0.0218,0.212,0.457,130.418,5
3,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,0,176907,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.321,0.0946,7,-27.961,1,0.0504,0.995,0.9180,0.104,0.397,169.980,3
4,08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,0,163080,0,['Dick Haymes'],['3BiJGZsyX9sJchTqcSA7Su'],1922,0.402,0.1580,3,-16.900,0,0.0390,0.989,0.1300,0.311,0.196,103.220,4


To cross-reference the songs from the charts to the Spotify data, we need the title and the artist to match. 

However, since the artists is in a string format resembling an array, we need to "evaluate" it and get the first artist to match it with the charts `DataFrame`.

In [13]:
tracks_df['artist'] = tracks_df['artists'].apply(lambda x: eval(x)[0])
tracks_df['artist']

0                     Uli
1         Fernando Pessoa
2         Ignacio Corsini
3         Ignacio Corsini
4             Dick Haymes
               ...       
586667            阿YueYue
586668         ROLE MODEL
586669            FINNEAS
586670       Gentle Bones
586671          Afrosound
Name: artist, Length: 586672, dtype: object

This methodology, however, might have some consequences that we might consider when we interpret our data in the succeeding notebooks. Some artist fields in the original dataset have artist features in them (more than one artists on the same track). 

Since we're only matching one artist in the Spotify data, we might not be able to merge them all. One way to address this is to separate the artists within a track. In other words, for every token that resembles more than one artist, we duplicate the entry with each artist on one record. In this way, we can properly match all songs in the first `DataFrame` to the second.

However, there are plenty ways to signify an artist feature (e.g. "Featuring.", "Feat.", "&", ",", ...). In the interest of time, we ignore the songs with artist features. Since we have a large sample size, we can justify this action via central limit theorem.



Another caveat in our augmented dataset is the duplicate tracks. Since Spotify tracks all versions of a track, the same song might have multiple records.

In [15]:
tracks_df[tracks_df.duplicated(subset=['name', 'artists'], keep=False)].sort_values('name').head()

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,artist
102819,6DdWA7D1o5TU2kXWyCLcch,"""40"" - Remastered 2008",37,157667,0,['U2'],['51Blml2LZPmy7TTiAg47vQ'],1983-02-28,0.371,0.545,...,-9.315,1,0.0307,0.185,0.582,0.183,0.310,150.316,4,U2
102846,3vMmwsAiLDCfyc1jl76lQE,"""40"" - Remastered 2008",38,157667,0,['U2'],['51Blml2LZPmy7TTiAg47vQ'],1983-02-28,0.371,0.545,...,-9.315,1,0.0307,0.185,0.582,0.183,0.310,150.316,4,U2
9222,5AZvNKWS9yPpXntKqHAhYw,"""Carmen Fantasie"", by Franz Waxman, based on t...",2,570920,0,"['Georges Bizet', 'Jascha Heifetz', 'Donald Vo...","['2D7RkvtKKb6E5UmbjQM1Jd', '7yaBUcdjmqPP2vIv6F...",1934,0.385,0.193,...,-20.298,1,0.0457,0.941,0.433,0.415,0.330,131.083,4,Georges Bizet
19915,0xVbtHC4BM071oCAYrMxy1,"""Carmen Fantasie"", by Franz Waxman, based on t...",0,583573,0,"['Georges Bizet', 'Jascha Heifetz', 'Donald Vo...","['2D7RkvtKKb6E5UmbjQM1Jd', '7yaBUcdjmqPP2vIv6F...",1946,0.289,0.198,...,-19.978,1,0.0539,0.943,0.376,0.194,0.342,70.255,4,Georges Bizet
267185,5AjpYr8ae9Us8ObS1r7LgB,"""Carmen Fantasie"", by Franz Waxman, based on t...",12,570467,0,"['Georges Bizet', 'Jascha Heifetz', 'Donald Vo...","['2D7RkvtKKb6E5UmbjQM1Jd', '7yaBUcdjmqPP2vIv6F...",1971,0.381,0.198,...,-19.838,1,0.0435,0.945,0.366,0.377,0.346,106.716,4,Georges Bizet


To address this, we just drop the duplicates since they more or less refer to the same song with similar features.

In [16]:
tracks_df.drop_duplicates(subset=['name', 'artists'], inplace=True)

We can combine all these features together in a single `DataFrame`.

In [17]:
final_df = cleaned_charts_df.merge(tracks_df[['name', 'artist', 'duration_ms', 'danceability', 'energy', 'loudness','mode', 'speechiness', 'acousticness', 'instrumentalness', 'valence']], left_on=['song', 'artist'], right_on=['name', 'artist'])
final_df

,song,artist,decade,weeks-on-board,rank,name,duration_ms,danceability,energy,loudness,mode,speechiness,acousticness,instrumentalness,valence
0,#1,Nelly,2000,20,22,#1,198760,0.690,0.592,-5.973,1,0.3000,0.0236,0.000000,0.466
1,#SELFIE,The Chainsmokers,2010,11,16,#SELFIE,183750,0.789,0.916,-3.262,1,0.2490,0.0135,0.000008,0.658
2,'65 Love Affair,Paul Davis,1980,20,6,'65 Love Affair,219813,0.647,0.686,-4.247,0,0.0274,0.4320,0.000006,0.952
3,'Til My Baby Comes Home,Luther Vandross,1980,16,29,'Til My Baby Comes Home,332227,0.804,0.714,-6.714,0,0.1830,0.0567,0.000006,0.802
4,'Til You Do Me Right,After 7,1990,22,31,'Til You Do Me Right,295000,0.663,0.462,-8.784,0,0.0252,0.3060,0.000000,0.431
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9337,i,Kendrick Lamar,2010,11,39,i,231933,0.755,0.884,-5.296,0,0.0638,0.0208,0.000003,0.897
9338,idontwannabeyouanymore,Billie Eilish,2010,3,96,idontwannabeyouanymore,203569,0.483,0.412,-8.461,1,0.0402,0.7370,0.000000,0.247
9339,sobeautiful,Musiq Soulchild,2000,11,84,sobeautiful,291133,0.689,0.562,-7.048,0,0.0637,0.0656,0.000000,0.210
9340,teachme,Musiq Soulchild,2000,20,42,teachme,260773,0.499,0.610,-5.110,0,0.1350,0.4730,0.000000,0.436


To use the same `DataFrame` in the future, we can simple serialize our object.

In [ ]:
final_df.to_pickle('./data/pkls/charts_with_audio_features_df.pkl')